In [2]:
import sys
sys.executable

'/opt/anaconda3/envs/DAIN/bin/python3'

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [8]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
GeForce RTX 2080 Ti, 460.39, 11019 MiB
GeForce RTX 2080 Ti, 460.39, 11019 MiB
GeForce RTX 2080 Ti, 460.39, 11016 MiB
GeForce RTX 2080 Ti, 460.39, 11019 MiB


In [2]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [4]:
################# Required Configurations ############################

#@markdown # Required Configuration
#@markdown Use the values in here to configure what you'd like DAIN to do.

#@markdown ## Input file
#@markdown Path (relative to the root of your Google Drive) to the input file. For instance, if you save your `example.mkv` file in your Google Drive, inside a `videos` folder, the path would be: `videos/example.mkv`. Currenly videos and gifs are supported.
INPUT_FILEPATH = "DAIN/input.mp4" #@param{type:"string"}

#@markdown ## Output file
#@markdown Output file path: path (relative to the root of your Google Drive) for the output file. It will also determine the filetype in the destination. `.mp4` is recommended for video input, `.gif` for gif inputs.
OUTPUT_FILE_PATH = "DAIN/output.mp4" #@param{type:"string"}

################# Optional configurations ############################

#@markdown # Optional Configuration
#@markdown Parameters below can be left with their defaults, but feel free to adapt them to your needs.

#@markdown ## Target FPS
#@markdown  how many frames per second should the result have. This will determine how many intermediate images are interpolated.
TARGET_FPS = 6 #@param{type:"number"}

#@markdown ## Frame input directory
#@markdown A path, relative to your GDrive root, where you already have the list of frames in the format 00001.png, 00002.png, etc.
FRAME_INPUT_DIR = '/mnt/wni_related_only/Sample/gsmap' #@param{type:"string"}

#@markdown ## Frame output directory
#@markdown A path, relative to your GDrive root, where you want the generated frame.
FRAME_OUTPUT_DIR = '/mnt/wni_related_only/Sample_output' #@param{type:"string"}

#@markdown ## Start Frame
#@markdown First frame to consider from the video when processing.
START_FRAME = 1 #@param{type:"number"}

#@markdown ## End Frame
#@markdown Last frame to consider from the video when processing. To use the whole video use `-1`.
END_FRAME = -1 #@param{type:"number"}

#@markdown ## Seamless playback
#@markdown Creates a seamless loop by using the first frame as last one as well. Set this to True this if loop is intended.
SEAMLESS = False #@param{type:"boolean"}

#@markdown ## Resize hotfix
#@markdown DAIN frames are a bit "shifted / smaller" compared to original input frames. This can partly be mitigated with resizing DAIN frames to the resolution +2px and cropping the result to the original resoultion with the starting point (1,1). Without this fix, DAIN tends to make "vibrating" output and it is pretty noticible with static elements like text.
#@markdown
#@markdown This hotfix tries to make such effects less visible for a smoother video playback. I do not know what DAINAPP uses as a fix for this problem, but the original does show such behaviour with the default test images. More advanced users can change the interpolation method. The methods cv2.INTER_CUBIC and cv2.INTER_LANCZOS4 are recommended. The current default value is cv2.INTER_LANCZOS4.
RESIZE_HOTFIX = True #@param{type:"boolean"}

#@markdown ## Auto-remove PNG directory
#@markdown Auto-delete output PNG dir after ffmpeg video creation. Set this to `False` if you want to keep the PNG files.
AUTO_REMOVE = True #@param{type:"boolean"}

In [ ]:
# # Interpolation
# %shell mkdir -p '{FRAME_OUTPUT_DIR}'
# %cd /content/DAIN

# !python -W ignore colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {frame_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}'

In [8]:
import time
import os
from torch.autograd import Variable
import torch
import numpy as np
import numpy
import networks
from my_args import args
from imageio import imread, imsave
from AverageMeter import  *
import shutil
import datetime
torch.backends.cudnn.benchmark = True

#args settings
args.netName = "DAIN_slowmotion"
args.time_step = 2/6
args.frame_input_dir = FRAME_INPUT_DIR
args.frame_output_dir = FRAME_OUTPUT_DIR


model = networks.__dict__[args.netName](
                                    channel = args.channels,
                                    filter_size = args.filter_size,
                                    timestep = args.time_step,
                                    training = False)

# --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {frame_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}

print(args.channels)



if args.use_cuda:
    model = model.cuda()

model_path = './model_weights/best.pth'
if not os.path.exists(model_path):
    print("*****************************************************************")
    print("**** We couldn't load any trained weights ***********************")
    print("*****************************************************************")
    exit(1)

if args.use_cuda:
    pretrained_dict = torch.load(model_path)
else:
    pretrained_dict = torch.load(model_path, map_location=lambda storage, loc: storage)

model_dict = model.state_dict()
# 1. filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
# 2. overwrite entries in the existing state dict
model_dict.update(pretrained_dict)
# 3. load the new state dict
model.load_state_dict(model_dict)
# 4. release the pretrained dict for saving memory
pretrained_dict = []

model = model.eval() # deploy mode

frames_dir = args.frame_input_dir
output_dir = args.frame_output_dir

timestep = args.time_step
time_offsets = [kk * timestep for kk in range(1, int(1.0 / timestep))]

input_frame = args.start_frame - 1
loop_timer = AverageMeter()

final_frame = args.end_frame

torch.set_grad_enabled(False)

# we want to have input_frame between (start_frame-1) and (end_frame-2)
# this is because at each step we read (frame) and (frame+1)
# so the last iteration will actuall be (end_frame-1) and (end_frame)
while input_frame < final_frame - 1:
    input_frame += 1

    start_time = time.time()

    filename_frame_1 = os.path.join(frames_dir, 'gsmap_nrt_japansub.20180601.0000.bin')
    filename_frame_2 = os.path.join(frames_dir, 'gsmap_nrt_japansub.20180601.0100.bin')
    
#     filename_frame_1 = os.path.join(frames_dir, f'{input_frame:0>5d}.bin')
#     filename_frame_2 = os.path.join(frames_dir, f'{input_frame+1:0>5d}.bin')

    X0 = torch.from_numpy(np.fromfile(filename_frame_1, dtype=np.float32).reshape(1,279,319) / 255.0).type(args.dtype)
    X1 = torch.from_numpy(np.fromfile(filename_frame_2, dtype=np.float32).reshape(1,279,319) / 255.0).type(args.dtype)

#     X0 = torch.from_numpy(np.transpose(imread(filename_frame_1), (2,0,1)).astype("float32") / 255.0).type(args.dtype)
#     X1 = torch.from_numpy(np.transpose(imread(filename_frame_2), (2,0,1)).astype("float32") / 255.0).type(args.dtype)

    assert (X0.size(1) == X1.size(1))
    assert (X0.size(2) == X1.size(2))

    intWidth = X0.size(2)
    intHeight = X0.size(1)
    channels = X0.size(0)
    if not channels == 3:
        print(f"Skipping {filename_frame_1}-{filename_frame_2} -- expected 3 color channels but found {channels}.")
        continue

    if intWidth != ((intWidth >> 7) << 7):
        intWidth_pad = (((intWidth >> 7) + 1) << 7)  # more than necessary
        intPaddingLeft = int((intWidth_pad - intWidth) / 2)
        intPaddingRight = intWidth_pad - intWidth - intPaddingLeft
    else:
        intPaddingLeft = 32
        intPaddingRight= 32

    if intHeight != ((intHeight >> 7) << 7):
        intHeight_pad = (((intHeight >> 7) + 1) << 7)  # more than necessary
        intPaddingTop = int((intHeight_pad - intHeight) / 2)
        intPaddingBottom = intHeight_pad - intHeight - intPaddingTop
    else:
        intPaddingTop = 32
        intPaddingBottom = 32

    pader = torch.nn.ReplicationPad2d([intPaddingLeft, intPaddingRight, intPaddingTop, intPaddingBottom])

    X0 = Variable(torch.unsqueeze(X0,0))
    X1 = Variable(torch.unsqueeze(X1,0))
    X0 = pader(X0)
    X1 = pader(X1)

    if args.use_cuda:
        X0 = X0.cuda()
        X1 = X1.cuda()

    y_s, offset, filter = model(torch.stack((X0, X1),dim = 0))
    y_ = y_s[args.save_which]

    if args.use_cuda:
        X0 = X0.data.cpu().numpy()
        if not isinstance(y_, list):
            y_ = y_.data.cpu().numpy()
        else:
            y_ = [item.data.cpu().numpy() for item in y_]
        offset = [offset_i.data.cpu().numpy() for offset_i in offset]
        filter = [filter_i.data.cpu().numpy() for filter_i in filter]  if filter[0] is not None else None
        X1 = X1.data.cpu().numpy()
    else:
        X0 = X0.data.numpy()
        if not isinstance(y_, list):
            y_ = y_.data.numpy()
        else:
            y_ = [item.data.numpy() for item in y_]
        offset = [offset_i.data.numpy() for offset_i in offset]
        filter = [filter_i.data.numpy() for filter_i in filter]
        X1 = X1.data.numpy()

    X0 = np.transpose(255.0 * X0.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))
    y_ = [np.transpose(255.0 * item.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight,
                                intPaddingLeft:intPaddingLeft+intWidth], (1, 2, 0)) for item in y_]
    offset = [np.transpose(offset_i[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0)) for offset_i in offset]
    filter = [np.transpose(
        filter_i[0, :, intPaddingTop:intPaddingTop + intHeight, intPaddingLeft: intPaddingLeft + intWidth],
        (1, 2, 0)) for filter_i in filter]  if filter is not None else None
    X1 = np.transpose(255.0 * X1.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))

    interpolated_frame_number = 0
    shutil.copy(filename_frame_1, os.path.join(output_dir, f"{input_frame:0>5d}{interpolated_frame_number:0>3d}.bin"))
    for item, time_offset in zip(y_, time_offsets):
        interpolated_frame_number += 1
        output_frame_file_path = os.path.join(output_dir, f"{input_frame:0>5d}{interpolated_frame_number:0>3d}.bin")
        
        item.astype('float32').tofile(output_frame_file_path)
        
#         imsave(output_frame_file_path, np.round(item).astype(numpy.uint8))
        # ให้เปลัี่ยนเป็น Save to file ด้วย numpy, ไม่ต้องใช้ np.round , astype เป็น float32 

    end_time = time.time()
    loop_timer.update(end_time - start_time)

    frames_left = final_frame - input_frame
    estimated_seconds_left = frames_left * loop_timer.avg
    estimated_time_left = datetime.timedelta(seconds=estimated_seconds_left)
    print(f"****** Processed frame {input_frame} | Time per frame (avg): {loop_timer.avg:2.2f}s | Time left: {estimated_time_left} ******************" )

# Copying last frame
last_frame_filename = os.path.join(frames_dir, str(str(final_frame).zfill(5))+'.bin')
shutil.copy(last_frame_filename, os.path.join(output_dir, f"{final_frame:0>5d}{0:0>3d}.bin"))

print("Finished processing images.")


usage: ipykernel_launcher.py [-h] [--debug] [--netName {DAIN,DAIN_slowmotion}]
                             [--datasetName {V,i,m,e,o,_,9,0,K,_,i,n,t,e,r,p} [{V,i,m,e,o,_,9,0,K,_,i,n,t,e,r,p} ...]]
                             [--datasetPath DATASETPATH]
                             [--dataset_split DATASET_SPLIT] [--seed SEED]
                             [--numEpoch NUMEPOCH] [--batch_size BATCH_SIZE]
                             [--workers WORKERS] [--channels {1,3}]
                             [--filter_size {2,4,5,6,51}] [--lr LR]
                             [--rectify_lr RECTIFY_LR] [--save_which {0,1}]
                             [--time_step TIME_STEP]
                             [--flow_lr_coe FLOW_LR_COE]
                             [--occ_lr_coe OCC_LR_COE]
                             [--filter_lr_coe FILTER_LR_COE]
                             [--ctx_lr_coe CTX_LR_COE]
                             [--depth_lr_coe DEPTH_LR_COE]
                             [--alpha ALP

SystemExit: 2

/opt/anaconda3/envs/DAIN/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# Finding DAIN Frames, upscaling and cropping to match original
# %cd {FRAME_OUTPUT_DIR}

if (RESIZE_HOTFIX):
  images = []
  for filename in os.listdir(FRAME_OUTPUT_DIR):
    img = cv2.imread(os.path.join(FRAME_OUTPUT_DIR, filename))
    filename = os.path.splitext(filename)[0]
    if(not filename.endswith('0')):
      dimensions = (img.shape[1]+2, img.shape[0]+2)
      resized = cv2.resize(img, dimensions, interpolation=cv2.INTER_LANCZOS4)
      crop = resized[1:(dimensions[1]-1), 1:(dimensions[0]-1)]
      cv2.imwrite(f"{filename}.png", crop)